<a href="https://colab.research.google.com/github/ritikkumar01111999/python3/blob/main/final_tag_prediction_store_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing all the required module here 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
import time
import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import hamming_loss
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
warnings.filterwarnings("ignore")

**importing file**

In [ ]:
#upload of new file via your device
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


In [ ]:
#uploading csv file 
data = pd.read_csv("/content/data.csv")

**Data Cleaning**

In [ ]:
# load stop words
# import nltk
nltk.download('stopwords')
nltk.download('wordnet')
stop_word = stopwords.words('english')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#function work to clean data and remove unwanted element from it 
def cleanig_data(dataset):
  from nltk.corpus import stopwords
  dataset.drop(columns=['Unnamed: 0'], inplace=True)
  dataset = dataset.dropna()
  # df['url'] = df['url'].str.lower()
  dataset['description'] = dataset['description'].str.lower()
  dataset['tags'] = dataset['tags'].str.lower()
  #remove of html content 
  def cleanHtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext
  #function to clean the word of any punctuation or special characters
  def cleanPunc(sentence): 
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned
  def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent
  
  token=ToktokTokenizer()
  lemma=WordNetLemmatizer()

  def lemitizeWords(text):
    words=token.tokenize(text)
    listLemma=[]
    for w in words:
        x=lemma.lemmatize(w, pos="v")
        listLemma.append(x)
    return ' '.join(map(str, listLemma))
  
  # dataset['description'] = dataset['description'].str.lower()
  dataset['description'] = dataset['description'].apply(cleanHtml)
  dataset['description'] = dataset['description'].apply(cleanPunc)
  dataset['description'] = dataset['description'].apply(keepAlpha)
  
  dataset['description'] = dataset['description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
  dataset['description'] = dataset['description'].apply(lambda x: lemitizeWords(x))
  return dataset

In [ ]:
data_result = cleanig_data(data)

**Tag Processing**

In [ ]:
# split tags in space
data_result['new_tags'] = data_result["tags"].apply(lambda x: x.split())

In [ ]:
# add all tags
all_tags = [item for sublist in data_result['new_tags'].values for item in sublist]

In [ ]:
cleaned_data = data_result.copy()

In [ ]:
cleaned_data.head()

,tags,description,new_tags
0,licence-needed supervising-job 5-plus-years-ex...,company employer midstream service provider on...,"[licence-needed, supervising-job, 5-plus-years..."
1,2-4-years-experience-needed salary full-time-job,icr staff accept resume industrial maintenance...,"[2-4-years-experience-needed, salary, full-tim..."
2,part-time-job,great position right person healthcareseeker c...,[part-time-job]
3,licence-needed,large multi specialty health center expand adu...,[licence-needed]
4,5-plus-years-experience-needed full-time-job b...,job purpose account director responsible manag...,"[5-plus-years-experience-needed, full-time-job..."


In [ ]:
cleaned_data.shape

(3504, 3)

**train - test Split**

In [ ]:
X = cleaned_data['description']
y = cleaned_data['new_tags']

**Encoding**

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
mulitlabel_binarizer = MultiLabelBinarizer()
y_bin = mulitlabel_binarizer.fit_transform(y)

In [ ]:
classes = mulitlabel_binarizer.classes_

In [ ]:
classes

array(['1-year-experience-needed', '2-4-years-experience-needed',
       '5-plus-years-experience-needed', 'associate-needed',
       'bs-degree-needed', 'full-time-job', 'hourly-wage',
       'licence-needed', 'ms-or-phd-needed', 'part-time-job', 'salary',
       'supervising-job'], dtype=object)

In [ ]:
# all_tags= vectorizer_count.get_feature_names()

In [ ]:

vectorizer = TfidfVectorizer(analyzer = 'word',
                                       min_df=0.0,
                                       max_df = 1.0,
                                       strip_accents = None,
                                       encoding = 'utf-8', 
                                       preprocessor=None,
                                       max_features=1000)

multilabel_x_data = vectorizer.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(multilabel_x_data, y_bin, test_size=0.3,random_state=123)

In [ ]:
print("Dimensions of train data X:",X_train.shape, "Y :",y_train.shape)
print("Dimensions of test data X:",X_test.shape,"Y:",y_test.shape)

Dimensions of train data X: (2452, 1000) Y : (2452, 12)
Dimensions of test data X: (1052, 1000) Y: (1052, 12)


**Training Model with linear SVC**

In [ ]:
classifier = OneVsRestClassifier(LinearSVC(verbose=0))
classifier.fit(X_train,y_train )
predictions = classifier.predict (X_test)
predictions_train = classifier.predict (X_train)

In [ ]:
y_true = y_test
y_logits = predictions

In [ ]:
def result_check(x, y):
     count = 0
     for i in range(len(x)):
         if x[i] == y[i]:
             count += 1
     result = count / len(x)
     return result

In [ ]:
cleaned_data['description'][3000]

'custom invitation design print company years solid performance seek offset press operator immediate need current pressman retire train right candidate experience would extremely beneficial must mechanically incline basic computer skills require must excellent visual color repeat ability job run short job volume moderate expect several color change typical day necessary minor maintenance equipment repair work contract knowledge thermography helpful necessary run single color ryobi envelope feeder head b dick duplicator thermography wide variety paper media biggest challenge position color match efficient must team player adaptable flexible detail orient ability follow write print instructions keep real physical demand job include lift stand day must able work minimal supervision train hours f overtime weekend need especially busy season november february rate pay base experience full benefit please submit cover letter indicate interest position resume'

In [ ]:
def subset_accuracy(y_true,y_predict,threshold):
  count = 0
  for j in range(len(y_true)):
    if result_check(y_true[j] , y_predict[j]) >= threshold:
      count = count + 1
  accuracy = count / len(y_true)
  return accuracy


In [ ]:
threshold_list = [1,.9,.8,.7,.6,.5]
for i in threshold_list:
  print(f'for threshold {i} accuracy is = ',subset_accuracy(y_true,y_logits,threshold=i))

for threshold 1 accuracy is =  0.18155893536121673
for threshold 0.9 accuracy is =  0.5009505703422054
for threshold 0.8 accuracy is =  0.7870722433460076
for threshold 0.7 accuracy is =  0.9496197718631179
for threshold 0.6 accuracy is =  0.9847908745247148
for threshold 0.5 accuracy is =  1.0


In [ ]:
print("train Accuracy :",metrics.accuracy_score(y_train, predictions_train))
print("test Accuracy :",metrics.accuracy_score(y_test, predictions))
print("Hamming loss ",metrics.hamming_loss(y_test,predictions))


precision = precision_score(y_test, predictions, average='micro')
recall = recall_score(y_test, predictions, average='micro')
f1 = f1_score(y_test, predictions, average='micro')
 
print("Micro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

precision = precision_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
f1 = f1_score(y_test, predictions, average='macro')
 
print("Macro-average quality numbers")
print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

train Accuracy : 0.5411908646003263
test Accuracy : 0.18155893536121673
Hamming loss  0.13315906210392903
Micro-average quality numbers
Precision: 0.6235, Recall: 0.4534, F1-measure: 0.5250
Macro-average quality numbers
Precision: 0.6364, Recall: 0.3797, F1-measure: 0.4408


## saving model##

In [ ]:
from sklearn.externals import joblib
joblib.dump(classifier , 'model.pkl')
joblib.dump(vectorizer , 'vectorizer.pkl')
# to match index with class name
def print_class(arr):
  cls_list = []
  for index, value in enumerate(arr):
    if value == 1:
      cls_list.append(classes[index])
  return cls_list
      # print(classes[index])   


**Loading Model and saving it into the csv file**

In [ ]:
# to match index with class name
def print_class(arr):
  cls_list = []
  for index, value in enumerate(arr):
    if value == 1:
      cls_list.append(classes[index])
  return cls_list
      # print(classes[index])   
vect = joblib.load('vectorizer.pkl')
model = joblib.load('model.pkl')
testing = "full time job for developer"
v = vect.transform([testing])
p = model.predict(v)
testing_result=print_class(p[0])

#storing result

import csv 
df = pd.DataFrame(columns=['Input','Prediction','true_result','similarity'])
def simlr(testing_result,true):
  count = 0
  for i in true:
    if i in testing_result:
      count= count+1
  return count/len(true)
  sm = simlr(testing_result,true)
  d = {
    "Input":[],
     "Prediction":[],
     "true_result":[],
     "similarity":[],

}
for i in range(3465,3474):
  testing = data['description'][i]

  v = vect.transform([testing])
  p = model.predict(v)
  testing_result=print_class(p[0])

  
  true = data['tags'][i].split()

  sm = simlr(testing_result,true)


  d['Input'].append(testing)
  d['Prediction'].append(testing_result)
  d['true_result'].append(true)
  d['similarity'].append(sm)
x = pd.DataFrame(d)
x.to_csv('final.csv',index=False)